In [1]:
import tweepy
import os
import json
import pandas as pd
from google.cloud import pubsub_v1

In [3]:
with open('../tweet_auth.json') as json_file:
    auth_creds = json.load(json_file)


# Variables that contain the credentials to access Twitter API
CONSUMER_KEY = auth_creds['consumer_key']
CONSUMER_SECRET = auth_creds['consumer_secret']
ACCESS_TOKEN = auth_creds['access_token']
ACCESS_SECRET = auth_creds['access_token_secret']


# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api


# Create API object
api = connect_to_twitter_OAuth()

In [4]:
# Test credentials to pull my latest tweets and save to csv
    
latest_tweets = api.user_timeline(screen_name = 'stevendumbe', count=5, tweet_mode="extended")
data = []
for tweet in latest_tweets:
    try:
        data.append({"user_name": tweet._json['user']['screen_name'], "text": tweet._json['retweeted_status']['full_text']})
    except KeyError:  # Not a Retweet
        data.append({"user_name": tweet._json['user']['screen_name'], "text": tweet._json['full_text']})
print(data)
df = pd.json_normalize(data)
print(df)
df.to_csv('my_tweets.csv',index=False)

[{'user_name': 'stevendumbe', 'text': 'Bringing back the memories when @patrick_brandl and @stevendumbe met @RamyAshour . Thank you Ramy #Yalla https://t.co/WfUYPmYgmq'}, {'user_name': 'stevendumbe', 'text': 'Congratulations to @imperialcollege  PhD student and my friend @stevendumbe for winning the #WorldCup with @EnglandDodge in NYC #ItsComingHome https://t.co/OvD6D23tA8'}, {'user_name': 'stevendumbe', 'text': "Good luck to my friend @stevendumbe representing @EnglandDodge in the World Dodgeball Championship in #NYC. A PhD at @imperialcollege isn't enough. #ItsComingHome Check out @BBCSport https://t.co/Zyf0uk5obq and @EveningStandard https://t.co/FLidZ4agH6"}, {'user_name': 'stevendumbe', 'text': "We met @RamyAshour and we're never going to fail #TheArtist @BOsquash @joeybarrington #ICSquashOnTour https://t.co/205XAUqDtH"}, {'user_name': 'stevendumbe', 'text': "@PSAWorldTour just started watching and heard you guys putting Kidderminster down, Kidderminster is where it's at!"}]
     

In [5]:
# Override the tweepy StreamListener class.
# Define on_status to extract and print processed form of tweets
# Define on_error to disconnect on certain status code

class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        payload={}
        payload["username"]=status.user.screen_name
        payload["timestamp"]=str(status.created_at)
        if hasattr(status, "retweeted_status"):  # Check if Retweet
            try:
                payload["text"]=status.retweeted_status.extended_tweet["full_text"]
            except AttributeError:
                payload["text"]=status.retweeted_status.text
        else:
            try:
                payload["text"]=status.extended_tweet["full_text"]
            except AttributeError:
                payload["text"]=status.text
        print(payload)
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [6]:
# Instantiate StreamListener class, authenticate, and stream filtered tweets

streamListener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener, tweet_mode='extended')

tags = ["#london"]
stream.filter(track=tags)

{'username': 'Sareeya_B', 'timestamp': '2020-11-29 16:10:08', 'text': 'ตำรวจอังกฤษจับกุมผู้ประท้วง #lockdown ควบคุมการระบาด #โควิด19 โดยเดินขบวนใน #กรุงลอนดอน ท้าทายคำสั่งตำรวจที่ห้ามชุมนุมเพื่อป้องกัน #COVID19 \n\nArrests made as anti-lockdown protesters march in #London, defying police warning to obey #Covid19 restrictions\n\n https://t.co/6twyrjFEkP'}
{'username': 'Offices_4U', 'timestamp': '2020-11-29 16:10:29', 'text': 'This #London business centre on the #KingsRoad caters to each clients’ needs. Located in the heart of Chelsea and surrounded by shops, cafes, and parks – the perfect environment for networking. The space features 24/7 key card access and high speed wifi: https://t.co/dBV8ahM9vo https://t.co/mKJVSSfMb7'}
{'username': 'senceneolsunki', 'timestamp': '2020-11-29 16:10:30', 'text': 'The Moldovan Stefan, whose life changed with the #IFLC, succeeded in finishing his music production program with first place in the music school named "Abbey Road Institute" in #London. Stef

KeyboardInterrupt: 